In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import time

import sys
sys.path.append('../MoitraRohatgi/')
sys.path.append('../')
import auditor_tools
import algorithms
import experiments
import examples
import our_experiments

In [2]:
X,Y,dim = our_experiments.LoadEubankFreshData()

In [3]:
algorithms.ols(X,Y,np.ones(len(Y)))[-1]

60.60362580853325

In [4]:
timer = time.time()
eubank_sol = auditor_tools.solve_regression_fractional(X,Y,intercept=False, time_limit=1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-04
set residual constraints
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 1
start solving
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1008 rows, 551 columns and 1008 nonzeros
Model fingerprint: 0x4ff7d702
Model has 48 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [6e-02, 1e+00]
  QLMatrix range   [6e-01, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1008 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve removed 1008 rows and 0 columns
Presolve time: 0.00s
Presolved: 6096 rows, 2064 columns, 18368 nonzeros
Presolved model has 1512 bilinear constraint(s)
V

In [5]:
eubank_sol_int=auditor_tools.solve_regression_integral(X, Y, intercept=False, time_limit=1,
                             warm_start=[1 if eubank_sol[-2][x].X>.9 else 0 for x in range(len(eubank_sol[-2]))],
                                 warm_start_ub=eubank_sol[3].ObjBound)

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 552 columns and 505 nonzeros
Model fingerprint: 0x3da16e7e
Model has 48 quadratic constraints
Variable types: 48 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [6e-02, 1e+00]
  QLMatrix range   [6e-01, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 5e+02]

User MIP start produced solution with objective 476 (0.01s)
Loaded user MIP start with objective 476

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 3184 rows, 5256 columns, 14336 nonzeros
Presolved model has 3136 SOS constraint(s)
Variable types: 3184 continuous, 2072 integer (2072 binary)

Root relaxation: objective 5.040000e+02, 47 iterations, 0.00 sec

In [6]:
algorithms.ols(X,Y,[x.X for x in eubank_sol_int[-2]])[-1]

-0.4108854399921711

In [17]:
print('Indices (in R) to remove')
for x in range(len(X)):
    if eubank_sol_int[-2][x].X==0: print(x+1)

Indices (in R) to remove
365
368
369
379
380
389
409
411
416
426
429
433
440
449
453
454
459
462
466
469
478
481
484
495
496
497
499
500


In [8]:
print('Total run time: ', time.time()-timer)

Total run time:  8.809056043624878


### Solutions obtained via different implementations of ZAM
We first run our implementation of the usual ZAM algorithm (upper bound of 33); then run our resolving implementation (upper bound of 29); MR22's resolving implementation does not run on this instance because after removing some number of entries Cov turns into a singular matrix.

In [9]:
timer = time.time()
t2,w2=auditor_tools.ZAMinfluence_upper_bound(X,Y)
print('time taken: ', time.time()-timer)

time taken:  0.9183611869812012


In [10]:
algorithms.ols(X,Y,w2)[-1]

-0.3726881955567478

In [11]:
print(len(Y)-t2)

33


In [12]:
timer = time.time()
t1,w1=auditor_tools.ZAMinfluence_resolving(X,Y)
print('time taken: ', time.time()-timer)

time taken:  1.2219831943511963


In [13]:
algorithms.ols(X,Y,w1)[-1]

-1.7045472787226572

In [14]:
print(len(Y)-t1)

29


In [16]:
## And our spectral certifier
timer = time.time()
print(auditor_tools.spectral_certify(X,Y))
print('time taken: ', time.time()-timer)

0.2193312453076213
time taken:  0.22248315811157227


In [17]:
## We also run the MR implementation of KZC21 to show that it crashes
Xflipped = copy.deepcopy(np.flip(X,axis=1))
algorithms.sensitivity(Xflipped,Y)

LinAlgError: Singular matrix

In [3]:
# we also run gurobi integral without a warm start to show that it is much slower
eubank_sol_int=auditor_tools.solve_regression_integral(X, Y, intercept=False, time_limit=10000,
                             warm_start=None,warm_start_ub=None)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-04
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10000
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 552 columns and 1 nonzeros
Model fingerprint: 0x6f732135
Model has 48 quadratic constraints
Variable types: 48 continuous, 504 integer (504 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [6e-02, 1e+00]
  QLMatrix range   [6e-01, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3184 rows, 5256 columns, 14336 nonzeros
Presolved model has 3136 SOS constraint(s)
Variable types: 3184 continuous, 2072 integer (2072 binary)

Root relaxation: objective 

 73569 49734  464.00000  159  305   70.00000  503.00000   619%   2.5 1081s
 74751 50399  451.00000  177  304   70.00000  503.00000   619%   2.5 1108s
 76138 52298  440.00000  193  314   70.00000  503.00000   619%   2.5 1138s
 78795 53599  434.00000  209  360   70.00000  503.00000   619%   2.5 1163s
H79364 53589                      71.0000000  503.00000   608%   2.5 1163s
H79491 53578                      72.0000000  503.00000   599%   2.5 1163s
 80676 53608  431.00000  214  356   72.00000  503.00000   599%   2.6 1166s
 80710 53856  431.00000  215  355   72.00000  503.00000   599%   2.6 1176s
 80966 54333  425.00000  221  343   72.00000  503.00000   599%   2.5 1200s
 81761 55589  421.00000  230  353   72.00000  503.00000   599%   2.5 1221s
*82626 55572             490      73.0000000  503.00000   589%   2.6 1221s
 83381 56531  417.00000  239  351   73.00000  503.00000   589%   2.6 1246s
 84628 57302  412.00000  245  345   73.00000  503.00000   589%   2.6 1269s
 85521 57892  407.00000  